# <b><span style="color:#ADF88B">Fix Concatenation Results</span></b>

Script to fix the concatenated result data frame. I recreate excatly how the audio files are concatenated, and exstracting the target words, target PER, target WER and 

In [67]:
import sys      
script_directory = '../'
sys.path.append(script_directory)
import self_made_functions as smf
import pandas as pd
import jiwer
import os

In [72]:
def concat_audio(df:pd.DataFrame, s:int=0, e:int=10, name = ''):
    concat_rows = df.iloc[s:e]  # Rows to concatenate
    input_lst = list(concat_rows.Word.values) # get input string

    target_wer_count = 0
    pron_lst = [] # list of pron scores to calcualte CER
    
    for row in concat_rows.itertuples():
        per_score = (row.pronScores.count('1')/len(row.pronScores.split(' ')))
        pron_lst.append(row.pronScores)
        if per_score != 0:
            target_wer_count += 1   
    
    target_wer = target_wer_count/len(input_lst) # Correct Word Error Rate (WER)
    
    pron_lst = (' '.join(pron_lst)) # String of all the 0 adn 1s
    target_per = pron_lst.count('1')/len(pron_lst) # Correct Character Error Rate (CER)

    audio_name = name + f'words_{s}_{e}.wav'
    return target_per, target_wer, input_lst, audio_name

# Later I can go thorug the transcriptions and se that they are properly stripped, and calculated CER and WER
def modify_transcriptions(df_csv:pd.DataFrame):
    df_csv['transcribed_word_striped'] = df_csv['Transcribed'].apply(
    lambda x: x.strip().lower().replace(".", "").replace(",", "").replace("!", "").replace("?", "") 
    if isinstance(x, str) else x)      
    df_csv['striped_CER'] = df_csv.apply(lambda x: jiwer.cer(x['Word'], x['transcribed_word_striped']), axis=1)
    return df_csv

In [74]:
df_assessment, _ = smf.get_correct_df()
df_assessment['id'] = df_assessment['File name'].apply(lambda x: x.split('_')[0])


# Test 1 :  Grouping by score. Sorted by speaker.
df_test_1 = df_assessment.sort_values(by=['id'], ascending=False) # Sorted by ID
df_test_1 = df_test_1.reset_index(drop=True)
df_test_1 = df_test_1.groupby('Score')# Grouped by score
df_test_1_name = 'scores_id_sorted'

# Test 2 :  Grouping by score. Random speakers.
df_test_2 = df_assessment.sample(frac=1).reset_index(drop=True)
df_test_2 = df_test_2.groupby('Score')
df_test_2_name = 'scores_id_mixed'

# Test 3 :  Grouping by speaker. No sorting.
df_test_3 = df_assessment.groupby('id')# Grouped by speaker
df_test_3_name = 'group_id_sorted_score'

# Test 4 :  No group. Everything in random order.
df_test_4 = df_assessment.sample(frac=1).reset_index(drop=True)
df_test_4_name = 'no_group_mixed'

# --------- Store Information ------------ #
halo_df = pd.DataFrame(columns=['PER', 
                                'WER', 
                                'input_words_list', 
                                'audio_name', 
                                'test_name',
                                'test'])

In [75]:
# Test 1 : Grouping by score. Sorted by speaker.
for idx, score_group in df_test_1:
    for i in range(0, len(score_group)//10,10):
        s, e = i, i + 10 # get start and end index
        target_per, wer, input_lst, audio_name = concat_audio(score_group, s, e)
        new_df = {'PER' : target_per, 
                'WER' : wer, 
                'input_words_list' : [input_lst], 
                'audio_name' : audio_name,
                'test_name' : df_test_1_name,
                'test' : 1} 
        new_df = pd.DataFrame(new_df)
        halo_df = pd.concat([halo_df, new_df], ignore_index=True)

/tmp/ipykernel_3599180/3318079430.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  halo_df = pd.concat([halo_df, new_df], ignore_index=True)


In [76]:
# Test 2 : Grouping by score. Random speakers.
for idx, score_group in df_test_2:
    for i in range(0, len(score_group)//10,10):
        s, e = i, i + 10 # get start and end index
        target_per, target_wer, input_lst, audio_name = concat_audio(score_group, s, e)
        new_df = {'PER' : target_per, 
                'WER' : target_wer, 
                'input_words_list' : [input_lst], 
                'audio_name' : audio_name,
                'test_name' : df_test_2_name,
                'test' : 2} 
        new_df = pd.DataFrame(new_df)
        halo_df = pd.concat([halo_df, new_df], ignore_index=True)

In [77]:
# Test 3 : Grouping by speaker. No sorting.
for idx, person in df_test_3:
    for i in range(0, len(score_group)//10,10):
        s, e = i, i + 10 # get start and end index
        speaker_id = person.iloc[0]['id']
        target_per, target_wer, input_lst, audio_name = concat_audio(score_group, s, e, name=f'{speaker_id}_')
        new_df = {'PER' : target_per, 
                'WER' : target_wer, 
                'input_words_list' : [input_lst], 
                'audio_name' : audio_name,
                'test_name' : df_test_3_name,
                'test' : 3} 
        new_df = pd.DataFrame(new_df)
        halo_df = pd.concat([halo_df, new_df], ignore_index=True)

In [78]:
# Test 4 : No group. Everything in random order.
for i in range(0, len(df_test_4)//10,10):
        s, e = i, i + 10 # get start and end index
        target_per, target_wer, input_lst, audio_name = concat_audio(df_test_4, s, e)
        new_df = {'PER' : target_per, 
                'WER' : target_wer, 
                'input_words_list' : [input_lst], 
                'audio_name' : audio_name,
                'test_name' : df_test_4_name,
                'test' : 4} 
        new_df = pd.DataFrame(new_df)
        halo_df = pd.concat([halo_df, new_df], ignore_index=True)

## Modify the current information CSV files

1) ~~Load the transcribed CSV~~
2) ~~Check that all the audio_names and input_words_list match~~
3) ~~Add the correct information to the CSV~~
4) Strip the transcriptions properly
5) ~~Fix update CER and WER~~
6) ~~Create a new file with the correct information~~
7) Check no transcriptions has NaN values

In [98]:
file_name='../3x10_Concatenation_information/nb-whisper-base-verbatim__concatenated_audio_information_group_id_sorted_score_v1.csv'
check_df = pd.read_csv(file_name)

check_df.isna().sum()   
# check_df['translated_string'].isna().sum()   
# NB! Ignore translated_CER, translated_WER. They are suposed to be NaN.

speaker_id                  0
score                       0
input_string                0
translated_string           3
translated_CER            104
translated_WER            104
target_CER                  0
target_CER_sum              0
target_WER                  0
length_deviation_words      0
audio_name                  0
audio_path                  0
trans_CER                   0
trans_WER                   0
dtype: int64

In [101]:
file_lst = os.listdir('../3x10_Concatenation_information')

for file in file_lst:
    # print(f'Working on file: {file}')
    # ----- Step 1 : Load the data ----- #
    df = pd.read_csv(f'../3x10_Concatenation_information/{file}')
    get_test = file.split('_information_')[1].split('_v1')[0]
    
    df = df.rename(columns={'target_CER': 'PER', 'target_WER': 'WER'})
    
    # ----- Step 1.1 : Check for NaN values ----- #
    nan = df['translated_string'].isna().sum()   
    if nan > 0:
        print(f'Error: {nan} NaN values in the translated_string column for file {file}.')  
    
    # ----- Step 2 & 3 : Merge the data ----- #
    same_df_split = halo_df[(halo_df['test_name'] == get_test) & 
                        (halo_df['audio_name'].isin(df['audio_name']))]
    if not len(same_df_split) == len(df):
        print(f'Error: Length of split data frames does not match for file {file}.\n{len(same_df_split)} vs {len(df)}')
    
    # Merging df with same_df_split on the 'audio_name' column
    merged_df = df.merge(same_df_split[['audio_name', 'PER', 'WER']], on='audio_name', how='left', suffixes=('', '_new'))

    # Updating the 'PER' column in df with the values from the merged DataFrame
    df['PER'] = merged_df['PER_new']
    df['WER'] = merged_df['WER_new']
    
    # Check for nans to ensure smooth merge
    if df['PER'].isnull().values.any():
        print(f'Error: NaN values in the PER column for file {file}.')
    elif df['WER'].isnull().values.any():
        print(f'Error: NaN values in the WER column for file {file}.')
        
    # ----- Step 4 : Strip the transcriptions ----- #
    df['translated_string'] = df['translated_string'].apply(lambda x: x.strip().lower().replace(".", "").replace(",", "").replace("!", "").replace("?", "") 
        if isinstance(x, str) else x)
    
    # ----- Step 5 : Calculate the CER & WER ----- #
    if nan == 0:
        df['translated_WER'] = df.apply(lambda x: jiwer.wer(x['input_string'], x['translated_string']), axis=1)
        df['translated_CER'] = df.apply(lambda x: jiwer.cer(x['input_string'], x['translated_string']), axis=1)
    else:
        df['translated_WER'] = 1.0
        df['translated_CER'] = 1.0
    
    # ----- Step 6 : Remove and rearrange columns ----- #
    df = df.drop(columns=['trans_CER', 'trans_WER', 'target_CER_sum'])

    # Check if speaker_id is in the columns
    if 'speaker_id' in df.columns:
        new_arangment = ['speaker_id', 'score',
                        'input_string', 'PER', 'WER',
                        'translated_string','translated_CER', 'translated_WER',
                        'length_deviation_words',	'audio_name',	'audio_path']
        df = df[new_arangment]
        
    
    
    # --- Extract and modify the model name for plotting --- # 
    model_name = file.split('__')[0] 
    if model_name.startswith('nb'):
        mn_split = model_name.split('-whisper')
        if not len(mn_split) == 2:
            print(f'Error: model name split failed. Len: {len(mn_split)}. Split: {mn_split}')
        else:
            model_name = 'NNL' + mn_split[1]
    else:
        model_name = model_name.capitalize()
        
    df['model_name'] = model_name
    
    # ----- Step 7 : Save the data ----- #
    dir = '../Concat_results/corrected_data'
    if not os.path.exists(dir):
        os.makedirs(dir)
        
    df.to_csv(f'{dir}/{file}', index=False)

Error: 3 NaN values in the translated_string column for file nb-whisper-base-verbatim__concatenated_audio_information_group_id_sorted_score_v1.csv.
Error: 1 NaN values in the translated_string column for file nb-whisper-base-verbatim__concatenated_audio_information_no_group_mixed_v1.csv.
Error: 2 NaN values in the translated_string column for file nb-whisper-base-verbatim__concatenated_audio_information_scores_id_sorted_v1.csv.
Error: 1 NaN values in the translated_string column for file nb-whisper-medium__concatenated_audio_information_scores_id_mixed_v1.csv.
Error: 1 NaN values in the translated_string column for file nb-whisper-medium-verbatim__concatenated_audio_information_scores_id_mixed_v1.csv.
